In [1]:
import pandas as pd
import re
import numpy as np
import plotly.graph_objects as go
import urllib, json
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

## Import data

In [2]:
folder = "C:/Users/samspencer/OneDrive - Royal Town Planning Institute/Documents/Projects/Prior Approvals/"

# Read in prior approval data from local directory

priors = 'Prior approvals for analysis 20211215'

df = pd.read_excel(folder + priors+'.xlsx')

### Filter out refused applications

In [3]:
#### Filter out refused applications

refusals = df[df['Status (groups)']=='Refused']

df = df[(df['Status (groups)']!= 'Refused ')&(df['Status (groups)']!= 'Refused')]

In [4]:
df['Planning authority'].value_counts()

Ealing                    4556
Richmond                  2042
Enfield                   1582
Barnet                    1278
Croydon                   1051
Havering                  1047
Bromley                   1004
Brent                     1001
Hillingdon                 887
Lewisham                   883
Merton                     763
Greenwich                  690
Sutton                     677
Harrow                     645
Newham                     554
Kingston                   498
Lambeth                    481
Bexley                     431
Wandsworth                 360
Hammersmith & Fulham       344
Hackney                    285
Tower Hamlets              281
Camden                     267
Southwark                  240
Hounslow                   224
Haringey                   215
Islington                  177
Redbridge                  170
Waltham Forest             120
Westminster                113
Kensington & Chelsea        40
Barking & Dagenham          32
City of 

In [5]:
# Kensington and Chelsea particularly low

## Clean Date Fields

In [ ]:
### Convert time to Month-Year

# Where the value for 'Decision date' is null, replace it with 'Valid date'
df['date'] = np.where(pd.to_datetime(df['Decision date']).isnull(),pd.to_datetime(df['Valid date']),pd.to_datetime(df['Decision date']))

df['month_year'] = df['date'].apply(lambda x: x.strftime('%B-%Y') if x > pd.to_datetime("2000-12-30") else None )

df['year'] =  df['date'].apply(lambda x: x.strftime('%Y') if x > pd.to_datetime("2000-12-30") else None )

## Clean Change of Use Fields

A significant amount of rows lack a value for 'application_type_full' so where those values are null, we need to replace it with the 'Assumed application type' value

In [ ]:
df['Application type on Datahub'].value_counts(dropna = False).head()

In [ ]:
df['Application type (assumed)'].value_counts(dropna = False).head()

In [ ]:
df['Application type on Datahub'] = df['Application type on Datahub'].apply(lambda x: None if isinstance(x,float) else x)

df['application_type_new'] = df.apply(lambda x: x['Application type on Datahub'] if x['Application type on Datahub'] else x['Application type (assumed)'], axis = 1)

In [ ]:
### Create a mapping schema to simplify names

type_map = {
    'retail/takeaway':'retail/service/takeaway/etc',
    'retail/service/etc':'retail/service/takeaway/etc',
    'Commercial, Business and Service':'commercial/business/service',
    'retail':'retail/service/takeaway/etc',
    'commercial/business/service/etc':'commercial/business/service',
    'mixed use including up':'commercial/business/service'
}

In [ ]:
# Most changes can be caught using a simple formula, splitting text patterns
# Additional elif clauses added for handling fringe cases

def split_change(text):
    changed_from = None
    changed_to = None
    if isinstance(text, str):
        if ' to ' in text and ' - ' in text:
            split = re.split(r' to ', text)
            changed_to = split[1]
            changed_from = re.split(r' - ',split[0])[1]
    # Handle fringe cases that may not follow above format
        elif text == 'Retail to residential':
            changed_to = 'dwellinghouses'
            changed_from = 'retail'
        elif text == 'Prior Approval: Change of use from Commercial, Business and Service (Use Class E) to Dwellinghouses (Use Class C3)':
            changed_to = 'dwellinghouses'
            changed_from = 'Commercial, Business and Service'
    if changed_to in type_map:
        changed_to = type_map[changed_to]
    if changed_from in type_map:
        changed_from = type_map[changed_from]
    return (changed_to, changed_from)      

In [ ]:
### Apply function to split change to before/after

df['changes'] = df['application_type_new'].apply(lambda x: split_change(x))

df['changed_from'] = df['changes'].apply(lambda x: x[1])

df['changed_to'] = df['changes'].apply(lambda x: x[0])

In [ ]:
df['changes'].head()

In [ ]:
non_null_df = df[df['changes']!= (None, None)]

In [ ]:
non_null_df['changes'].value_counts()

In [ ]:
# Quick function to clean up the formatting of the to/from labels
# Important bit is to add the extra space to some labels (either to or from) to make it different, otherwise Sankey chart will be distorted


def clean_labels(x):
    to = x[0]
    fr = x[1]
    to = to.capitalize()
    fr = fr.capitalize()
    to += ' '
    return (to, fr)

### All Changes

In [ ]:
change_df = pd.DataFrame(non_null_df['changes'].value_counts()).reset_index()
change_df['index'] = change_df['index'].apply(lambda x: clean_labels(x))
change_df['changed_to'] = change_df['index'].apply(lambda x: x[0])
change_df['changed_from'] = change_df['index'].apply(lambda x: x[1])
change_df

In [ ]:
types = list(set(list(change_df['changed_to'])+list(change_df['changed_from'])))

value_dict = dict(zip(change_df['index'], change_df['changes']))

fig = go.Figure(data=[go.Sankey(
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
        label = types
    ),
    # Add links
    link = dict(
      source =  [types.index(x) for x in change_df['changed_from']],
      target =  [types.index(x) for x in change_df['changed_to']],
      value =  [value_dict[x] for x in zip(change_df['changed_to'],change_df['changed_from'])],
    label = [value_dict[x] for x in zip(change_df['changed_to'],change_df['changed_from'])]))])
    #  color =  data['data'][0]['link']['color']
#))])

fig.update_layout(title_text="Prior Approvals - Change of Use",font_size=10)
fig.show()

### Changes in 2021

In [ ]:

change_df = pd.DataFrame(non_null_df[non_null_df['date']>=pd.to_datetime('2021')]['changes'].value_counts()).reset_index()
change_df['index'] = change_df['index'].apply(lambda x: clean_labels(x))
change_df['changed_to'] = change_df['index'].apply(lambda x: x[0])
change_df['changed_from'] = change_df['index'].apply(lambda x: x[1])
change_df

In [ ]:
types = list(set(list(change_df['changed_to'])+list(change_df['changed_from'])))

value_dict = dict(zip(change_df['index'], change_df['changes']))

fig = go.Figure(data=[go.Sankey(
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 50,
      line = dict(color = "black", width = 0.5),
        label = types
    ),
    # Add links
    link = dict(
      source =  [types.index(x) for x in change_df['changed_from']],
      target =  [types.index(x) for x in change_df['changed_to']],
      value =  [value_dict[x] for x in zip(change_df['changed_to'],change_df['changed_from'])],
    label = [value_dict[x] for x in zip(change_df['changed_to'],change_df['changed_from'])]))])
    #  color =  data['data'][0]['link']['color']
#))])

fig.update_layout(title_text="Prior Approvals - Change of Use - 2021",font_size=10)
fig.show()

## Residential Conversions

In [ ]:
## Use "decision_date" or "valid_date" ?

## Combination of the two where 

In [ ]:
### Flag Residential Conversions

# May need to change this logic later if conversion name changes

df['residential_conversion'] = df['changed_to'].apply(lambda x: True if x == "dwellinghouses" else False)

In [ ]:
### Flag whether on a High Street or in a City Centre

def isNaN(x):
    return x != x

df['High Street'] = df['High Street name'].apply(lambda x: 0 if isNaN(x) else 1)

df['Town Centre'] = df['Town Centre Name'].apply(lambda x: 0 if isNaN(x) else 1)

In [ ]:
df[['High Street name','High Street']].head()

In [ ]:
### Monthly residential conversions

residential_converstions_monthly = df[df['residential_conversion']==True].groupby('month_year').agg({'id':'count','Residential units proposed':'sum','Existing non-residential floorspace':'median','High Street':'sum','Town Centre':'sum'})

residential_converstions_monthly = pd.DataFrame(residential_converstions_monthly).reset_index()
residential_converstions_monthly['month_year'] = pd.to_datetime(residential_converstions_monthly['month_year'], format = "%B-%Y")
residential_converstions_monthly = residential_converstions_monthly.sort_values('month_year')


# Monthly proportion of high street and town centre prior approvals

residential_converstions_monthly['Pct High Street'] = residential_converstions_monthly['High Street']/residential_converstions_monthly['id']
residential_converstions_monthly['Pct Town Centre'] = residential_converstions_monthly['Town Centre']/residential_converstions_monthly['id']
residential_converstions_monthly =residential_converstions_monthly.dropna()
#residential_converstions_monthly['month_year'] = residential_converstions_monthly['month_year'].apply(lambda x: x.strftime('%B-%Y') if x > pd.to_datetime("2000-12-30") else None)

In [ ]:
### Annual residential conversions

residential_converstions_annual = df[df['residential_conversion']==True].groupby('year').agg({'id':'count','Residential units proposed':'sum','Existing non-residential floorspace':'median','High Street':'sum','Town Centre':'sum'})
residential_converstions_annual = pd.DataFrame(residential_converstions_annual).reset_index()
residential_converstions_annual['year'] = pd.to_datetime(residential_converstions_annual['year'], format = "%Y")
residential_converstions_annual = residential_converstions_annual.sort_values('year')

residential_converstions_annual['Pct High Street'] = residential_converstions_annual['High Street']/residential_converstions_annual['id']
residential_converstions_annual['Pct Town Centre'] = residential_converstions_annual['Town Centre']/residential_converstions_annual['id']

residential_converstions_annual =residential_converstions_annual.dropna()

#### Total prior approval residential conversions by month

In [ ]:
# Doesn't appear that there have been a huge spike in residential converstions after August--but would be expecting a jump
# in August? Or just 2021?

# 

plt.plot(residential_converstions_monthly['month_year'],residential_converstions_monthly['id'])
plt.title('Total Prior Approval Residential Conversions by Month')
plt.show()

#### Total prior approval residential conversions by year

In [ ]:
# Doesn't appear that there have been a huge spike in residential converstions after August--but would be expecting a jump
# in August? Or just 2021?

# 

plt.plot(residential_converstions_annual['year'],residential_converstions_annual['id'])
plt.title('Total Prior Approval Residential Conversions by Year')
plt.show()

#### Median non-residential floorspace by month

In [ ]:
# A few months look like outliers--group annually to smooth? -- June, July, Nov 2020
# This is due to null values in the floorspace field and only a few large values

plt.plot(residential_converstions_monthly['month_year'],residential_converstions_monthly['Existing non-residential floorspace'])
plt.title('Median Non-residential Floorspace by Month')
plt.show()

#### Median non-residential floor space by year

In [ ]:
# A few months look like outliers--group annually to smooth? -- June, July, Nov 2020

# Would it be possible assume that the average floorspace per unit is simply this column divided by the 

plt.plot(residential_converstions_annual['year'],residential_converstions_annual['Existing non-residential floorspace'])
plt.title('Median Non-residential Floorspace by Year')
plt.show()

#### Sum of 'Residential units proposed' by month

In [ ]:
# Why 0 units in most recent months?

plt.plot(residential_converstions_monthly['month_year'],residential_converstions_monthly['Residential units proposed'])
plt.title('Sum of Residential units proposed by month')
plt.show()

In [ ]:
residential_converstions_monthly.tail(20)

#### Sum of 'Residential units proposed' by year

In [ ]:
# Have I set myself up so that I'm using incomplete data? 
plt.plot(residential_converstions_annual['year'],residential_converstions_annual['Residential units proposed'])
plt.title('Sum of Residential units proposed by year')
plt.show()

In [ ]:
residential_converstions_monthly

#### Proportion of Prior Approvals on High Streets or Town Centres

In [ ]:

plt.plot(residential_converstions_monthly['month_year'],residential_converstions_monthly['Pct High Street'])
plt.plot(residential_converstions_annual['year'],residential_converstions_annual['Pct High Street'], color ='orange', linestyle = 'dashed')
plt.title('Proportion of Prior Approvals on High Streets or Town Centres')
#plt.legend(['Percent on Town Centres - Monthly','Percent on High Streets - Monthly','Percent on Town Centres - Annual','Percent on High Streets - Annual'])
plt.show()

In [ ]:
plt.plot(residential_converstions_monthly['month_year'],residential_converstions_monthly['Pct Town Centre'])
plt.plot(residential_converstions_annual['year'],residential_converstions_annual['Pct Town Centre'], color ='b', linestyle = 'dashed')
plt.title('Proportion of Prior Approvals on Town Centres')
plt.show()

In [ ]:
plt.plot(residential_converstions_monthly['month_year'],residential_converstions_monthly['Pct High Street'])
plt.plot(residential_converstions_annual['year'],residential_converstions_annual['Pct High Street'], color ='b', linestyle = 'dashed')
plt.title('Proportion of Prior Approvals on High Streets')
plt.show()

### Map Changes over time

In [ ]:
### Hold off on mapping for now

In [ ]:
gdf = gpd.read_file(folder+'Local_Planning_Authorities_(April_2019)_UK_BUC.geojson')

In [ ]:
gdf.head()

In [ ]:
df['Planning authority'].unique()

In [ ]:
gdf['lpa19nm'].unique()

In [ ]:
gdf.plot()